In [13]:
# %load test.py
"""
1D KF Simulation using Filterpy

2018-01-24 msrks
"""

import numpy as np
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

FRAME_RATE = 25
DT = 1/FRAME_RATE
VAR = 0.1 

f = KalmanFilter(dim_x=2, dim_z=1)

# init state
f.x = np.array([[1.],
                [0.]])       
# init covariance matrix
f.P *= 1000.                 

# state transition matrix
f.F = np.array([[1.,DT], # 1じゃなくて dtだと思われるので修正
                [0.,1.]])    
# measurement matrix
f.H = np.array([[1.,0.]])    

# prediction noise
f.Q = Q_discrete_white_noise(2, dt=DT, var=VAR)
# observation noise
f.R = 5 

# >>> f.Q = Q_discrete_white_noise(2, dt=1.0, var=0.1)
# >>> print (f.Q)
# [[0.025 0.05 ]
#  [0.05  0.1  ]]

print ("Initial State:")
print (f.x)

meas = [2,4,6,8,10]
# loop
for t in range(len(meas)):
    print (f"\nTime {t}:")
    f.predict()
    print (f.x)
    f.update(meas[t])
    print (f.x)

Initial State:
[[1.]
 [0.]]

Time 0:
[[1.]
 [0.]]
[[1.99503278]
 [0.03973773]]

Time 1:
[[1.99662229]
 [0.03973773]]
[[3.1356194 ]
 [6.97814121]]

Time 2:
[[3.41474505]
 [6.97814121]]
[[ 4.78089515]
 [19.58200693]]

Time 3:
[[ 5.56417543]
 [19.58200693]]
[[ 6.8471255 ]
 [30.80455998]]

Time 4:
[[ 8.0793079 ]
 [30.80455998]]
[[ 9.04816802]
 [38.10743631]]


## Q_discrete_white_noise のパラメーター varの決め方

<div class="alert alert-success">
var = 4*D/(T**4) と決めると良い！！！
</div>

#### 証明

T時刻後に、Dメートルくらバラついているとすると

In [4]:
T = 10
D = 1

$$D = (0.5T^2)^2\cdot var = 0.25T^4\cdot var$$

$$var = \frac{D}{0.25T^4} = \frac{4D}{T^4}$$

In [10]:
var = 4*D/(T**4)
var

0.0004

In [11]:
f.Q = Q_discrete_white_noise(2, dt=T, var=var)
f.Q

array([[1.  , 0.2 ],
       [0.2 , 0.04]])